In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn import svm
from sklearn.model_selection import cross_val_score
from sklearn.feature_extraction import text 
from sklearn.metrics import precision_recall_fscore_support
from sklearn import metrics

In [2]:
# import sys
# !conda install --yes --prefix {sys.prefix} numpy
# !conda install --yes --prefix {sys.prefix} scikit-learn
# !conda install --yes --prefix {sys.prefix} sklearn

In [3]:
##### to print multiple outputs from same cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [4]:
file_path = "/Users/tejasvibelsare/Documents/New_gold.xlsx"
train = pd.read_excel(file_path, sheet_name='Sheet1',usecols = "B,H")

In [5]:
len(train)

86

In [6]:
train.columns

Index(['Contents', 'category'], dtype='object')

In [7]:
len(train[train.category == 'allergic'])

45

In [8]:
len(train[train.category == 'infectious'])

41

In [9]:
train.head()

,Contents,category
0,My allergies killin my widdle eyes ??,allergic
1,Well think I woke up today with pink eye smh...,infectious
2,Seriously fuck allergies my eyes hurt and itch...,allergic
3,My eye itchin #allergies,allergic
4,November was hand foot and mouth. December is ...,infectious


In [10]:
## to convert all infectious records to 1 and allergic records to 0
# train.loc[train["category"]=='infectious',"category"]=1
# train.loc[train["category"]=='allergic',"category"]=0

from sklearn import preprocessing

encode = preprocessing.LabelEncoder()
train['category'] = encode.fit_transform(train['category'])

In [11]:
train.head()

,Contents,category
0,My allergies killin my widdle eyes ??,0
1,Well think I woke up today with pink eye smh...,1
2,Seriously fuck allergies my eyes hurt and itch...,0
3,My eye itchin #allergies,0
4,November was hand foot and mouth. December is ...,1


In [12]:
train_x = train["Contents"]
train_y = train["category"]
train_y=train_y.astype('int')
# train_y.todense()

In [13]:
train_x

0                 My allergies killin my widdle eyes ??
1       Well think I woke up today with pink eye smh...
2     Seriously fuck allergies my eyes hurt and itch...
3                              My eye itchin #allergies
4     November was hand foot and mouth. December is ...
5                      Allergies ugh ((MY EYES))!!!!!!!
6                                I have pink eye. Cool.
7                                   I have pink eye fml
8     Know how to feel like a kid again? Get a case ...
9                Allergy season??...my eyes itch so bad
10                       I woke up with Pink Eye wtffff
11                MY ALLERGIES OMG MY EYES I CAN'T DEAL
12    I have pink eye and a kid threw a pencil at me...
13                      Pretty sure I have pink eye. Tf
14    Thanks, Timehop, for reminding me that my kid ...
15                         Ironic I would have pink eye
16                            That kid gave me pink eye
17                         I have pink eye. Awes

In [14]:
tfidf = TfidfVectorizer(min_df=2,stop_words='english')

In [15]:
stop_words = text.ENGLISH_STOP_WORDS
len(stop_words)

318

In [16]:
stop_words

frozenset({'a',
           'about',
           'above',
           'across',
           'after',
           'afterwards',
           'again',
           'against',
           'all',
           'almost',
           'alone',
           'along',
           'already',
           'also',
           'although',
           'always',
           'am',
           'among',
           'amongst',
           'amoungst',
           'amount',
           'an',
           'and',
           'another',
           'any',
           'anyhow',
           'anyone',
           'anything',
           'anyway',
           'anywhere',
           'are',
           'around',
           'as',
           'at',
           'back',
           'be',
           'became',
           'because',
           'become',
           'becomes',
           'becoming',
           'been',
           'before',
           'beforehand',
           'behind',
           'being',
           'below',
           'beside',
           'besides'

In [17]:
# x_traintfidf = tfidf.fit_transform(x_train)
x_traintfidf = tfidf.fit_transform(train_x)
x_traintfidf.ndim
type(x_traintfidf)


2

scipy.sparse.csr.csr_matrix

In [18]:
len(tfidf.get_feature_names())
type(tfidf.get_feature_names())

train['features'] = list(x_traintfidf.toarray())

# train['names'] = []

# for i, row in train.iterrows():
#     train['names'][i] = tfidf.inverse_transform(x_traintfidf[i])
    
train
    

55

list

,Contents,category,features
0,My allergies killin my widdle eyes ??,0,"[0.7020298442682317, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,Well think I woke up today with pink eye smh...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Seriously fuck allergies my eyes hurt and itch...,0,"[0.19065492203115716, 0.0, 0.0, 0.407943221622..."
3,My eye itchin #allergies,0,"[0.7655598000081454, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,November was hand foot and mouth. December is ...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,Allergies ugh ((MY EYES))!!!!!!!,0,"[0.35577218270429944, 0.0, 0.0, 0.0, 0.0, 0.0,..."
6,I have pink eye. Cool.,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.884..."
7,I have pink eye fml,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,Know how to feel like a kid again? Get a case ...,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.6151106204552..."
9,Allergy season??...my eyes itch so bad,0,"[0.0, 0.4686167961676387, 0.0, 0.4918708754032..."


In [19]:
x_traintfidf.toarray()

array([[0.70202984, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.48368392,
        0.        ],
       [0.19065492, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.47198183, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27070981, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [20]:
tfidf.get_feature_names()
### list of all features



['allergies',
 'allergy',
 'baby',
 'bad',
 'breathe',
 'camp',
 'case',
 'child',
 'cool',
 'damn',
 'drops',
 'eye',
 'eyes',
 'fml',
 'fuck',
 'gave',
 'got',
 'home',
 'hurt',
 'infection',
 'itch',
 'itching',
 'just',
 'kid',
 'kids',
 'like',
 'morning',
 'need',
 'nose',
 'outside',
 'pink',
 'pissed',
 'poor',
 'pretty',
 'puffy',
 'screw',
 'season',
 'seriously',
 'smh',
 'sneezing',
 'son',
 'summer',
 'sure',
 'thank',
 'thanks',
 'think',
 'throat',
 'time',
 'today',
 'tonight',
 'ugh',
 'viral',
 'watering',
 'woke',
 'wtf']

In [21]:
mutual = SelectKBest(mutual_info_classif, k=50)
mutual_X_train = mutual.fit_transform(x_traintfidf, train_y)

In [22]:
mutual_X_train.toarray()

array([[0.70202984, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.48368392,
        0.        ],
       [0.19065492, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.47198183, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27070981, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [23]:
train_x[2]



'Seriously fuck allergies my eyes hurt and itch so bad! ??'

In [24]:
# tfidf.inverse_transform(mutual_X_train[1])
tfidf.inverse_transform(x_traintfidf[2])

[array(['allergies', 'eyes', 'seriously', 'fuck', 'hurt', 'itch', 'bad'],
       dtype='<U9')]

In [25]:
## all training data converted to vectors
arr = x_traintfidf.toarray()
arr.ndim


2

In [26]:
x_traintfidf
x_traintfidf.toarray()
TO_REMOVE = [1]
to_keep = list(set(range(x_traintfidf.shape[1]))-set(TO_REMOVE))    
new_x_traintfidf = x_traintfidf[:,to_keep]

new_x_traintfidf

<86x55 sparse matrix of type '<class 'numpy.float64'>'
	with 346 stored elements in Compressed Sparse Row format>

array([[0.70202984, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.48368392,
        0.        ],
       [0.19065492, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.47198183, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.27070981, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

<86x54 sparse matrix of type '<class 'numpy.float64'>'
	with 341 stored elements in Compressed Sparse Row format>

In [27]:
modelSVM = svm.SVC(kernel='linear')

In [28]:
train_y

0     0
1     1
2     0
3     0
4     1
5     0
6     1
7     1
8     1
9     0
10    1
11    0
12    1
13    1
14    1
15    1
16    1
17    1
18    0
19    1
20    0
21    0
22    0
23    0
24    0
25    0
26    1
27    1
28    0
29    0
     ..
56    0
57    1
58    0
59    0
60    1
61    1
62    0
63    1
64    1
65    0
66    1
67    0
68    1
69    1
70    0
71    1
72    1
73    0
74    0
75    1
76    1
77    0
78    0
79    0
80    1
81    0
82    0
83    0
84    1
85    0
Name: category, Length: 86, dtype: int64

In [29]:

# modelSVM_crossValidation_mutual_accuracy = cross_val_score(modelSVM,mutual_X_train,train_y,cv=5,scoring='accuracy')
modelSVM_crossValidation_mutual_accuracy = cross_val_score(modelSVM,new_x_traintfidf,train_y,cv=5,scoring='accuracy')
# modelSVM_crossValidation_mutual_f1 = cross_val_score(modelSVM,mutual_X_train,train_y,cv=5,scoring='f1')
modelSVM_crossValidation_mutual_f1 = cross_val_score(modelSVM,new_x_traintfidf,train_y,cv=5,scoring='f1')

In [30]:
modelSVM_crossValidation_mutual_accuracy

array([1.        , 1.        , 1.        , 0.94117647, 0.94117647])

In [31]:
modelSVM_crossValidation_mutual_f1

array([1.        , 1.        , 1.        , 0.94117647, 0.94117647])

In [32]:
modelSVM_crossValidation_mutual_accuracy.mean()

0.9764705882352942

In [33]:
modelSVM_crossValidation_mutual_f1.mean()

0.9764705882352942

In [34]:
### Fitting SVM model 


# modelSVM.fit(mutual_X_train, train_y)
modelSVM.fit(x_traintfidf, train_y)
coef = modelSVM.coef_[0]
coef.toarray()

SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
  kernel='linear', max_iter=-1, probability=False, random_state=None,
  shrinking=True, tol=0.001, verbose=False)

array([[-1.9010354 , -0.79066443,  0.45812634, -0.45071857, -0.43433437,
         0.24047185,  0.3418208 ,  0.41257055,  0.20288901, -0.42972954,
        -0.49814492,  0.641592  , -0.70394943,  0.33849307, -0.35417573,
         0.20317501, -0.0547295 ,  0.18414788, -0.17282191,  0.23944457,
        -0.32202759, -0.78791684,  0.19506824,  0.38776098,  0.51041841,
         0.12678189,  0.14464223, -0.44644992, -0.27731832, -0.56297538,
         1.97015591,  0.34653331,  0.22367191,  0.3173552 , -0.60283015,
        -0.17692284, -0.0760861 ,  0.20363708,  0.1330179 ,  0.        ,
         0.34684897,  0.24047185,  0.16049492,  0.41463574,  0.09522681,
         0.54757712, -0.13103593, -0.00636633,  0.19137176,  0.06942521,
        -0.00276279,  0.26241412, -0.42525023,  0.3054376 ,  0.47992846]])

In [35]:
file_path = "/Users/tejasvibelsare/Documents/Twitter10K.xlsx"
test = pd.read_excel(file_path, sheet_name='Sheet1',usecols = "B")
len(test)

4799

In [36]:
file_path = "/Users/tejasvibelsare/Documents/Twitter10K.xlsx"
classified_by_crimson= pd.read_excel(file_path, sheet_name='Sheet1',usecols = "L")
classified = classified_by_crimson["classified"]
len(classified)

4799

In [37]:
test.head()
test_x = test["Contents"]
len(test["Contents"])
# classified_by_crimson = test["classified"]

,Contents
0,"So I'm finally feeling better, just have a bad..."
1,I have pink eye in both eyes...this blows
2,"Some girl had pink-eye in my school today, and..."
3,I bet people think I'm high but I actually jus...
4,"@150CENTIMETRES hahahahaha me too, i got an mc..."


4799

In [38]:
test_x

0       So I'm finally feeling better, just have a bad...
1               I have pink eye in both eyes...this blows
2       Some girl had pink-eye in my school today, and...
3       I bet people think I'm high but I actually jus...
4       @150CENTIMETRES hahahahaha me too, i got an mc...
5            I'm pretty sure I have fucking pink eye. FML
6       @maggieenglish @emjs44 i would not suggest get...
7       I watched the vlog where @rj4gui4r got pink ey...
8        @ktrenkema I gots pink eye in both of my eyes ??
9       @JDRAMIREZ I just had to put eye allergy drops...
10                                  i have an eye allergy
11      Conjunctivitis in my eye :(   </3\n\n#FML #wor...
12      i couldn't go to tennis tryouts bc i have pink...
13                                    I have pink eye ugh
14      @samanthaleigh_5 ouch. I'm trying to get ready...
15               Oh my Lord my eyes itch!!! ?? #allergies
16                    My mom mad assl she has pink eye ??
17            

In [39]:
x_traintfidf.ndim

2

In [40]:
testtfidf = tfidf.transform(test_x)
testtfidf.shape

(4799, 55)

In [41]:
##### Finalizing features of test data

X_test = mutual.transform(testtfidf)
X_test.ndim

2

In [42]:
test_arr = X_test.toarray()
test_arr.shape

(4799, 50)

In [43]:
# pred_SVM = modelSVM.predict(X_test)
pred_SVM = modelSVM.predict(testtfidf)

In [44]:
SVM_accuracy = metrics.accuracy_score(classified, pred_SVM)

In [45]:
SVM_accuracy

0.999166493019379

In [46]:
#####SVM
precision, recall, F1, _ = list(precision_recall_fscore_support(classified, pred_SVM, beta=1.0, average="binary"))
precision , recall , F1

(0.9903381642512077, 0.9903381642512077, 0.9903381642512077)